In [ ]:
# =============================================================================
# EXPERIMENT: Gradient Boosting (Conservative)
# Date: 28/11 22:56
# Description: Gradient Boosting with conservative parameters to avoid overfitting
# eerste op meest advanced preprocessed dataset van 28/11 (advanced preprocessing met tips van ta) 6de versie
# =============================================================================

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import warnings
from datetime import datetime
from pathlib import Path
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold


In [2]:
# =============================================================================
# 1. LOAD PREPROCESSED DATA (no need to preprocess again!)
# =============================================================================
print("Loading preprocessed data...")
X = pd.read_pickle('../../data/processed/X_train_processed.pkl')
y = pd.read_pickle('../../data/processed/y_train.pkl')
X_test = pd.read_pickle('../../data/processed/X_test_processed.pkl')
test_ids = pd.read_pickle('../../data/processed/test_ids.pkl')

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

Loading preprocessed data...
X shape: (20885, 95)
y shape: (20885,)


In [3]:
# =============================================================================
# 2. DEFINE EXPERIMENT PARAMETERS
# =============================================================================
EXPERIMENT_NAME = "gradient_boosting_conservative"
MODEL_DESCRIPTION = "Gradient Boosting with conservative parameters"

In [4]:
# =============================================================================
# 3. HYPERPARAMETER TUNING
# =============================================================================
from sklearn.ensemble import GradientBoostingClassifier


print(f"\n{'='*70}")
print(f"EXPERIMENT: {EXPERIMENT_NAME}")
print(f"{'='*70}")

# Conservative parameters to prevent overfitting
param_dist = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],  # Lower = less overfitting
    'max_depth': [3, 4, 5],  # Shallow trees
    'min_samples_split': [20, 50, 100],
    'min_samples_leaf': [10, 20, 30],
    'subsample': [0.7, 0.8, 0.9],  # Use subset of data
    'max_features': ['sqrt', 'log2', 0.5]
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("\nRunning RandomizedSearchCV (trying 30 combinations)...")
print("This may take 15-25 minutes...\n")

random_search = RandomizedSearchCV(
    GradientBoostingClassifier(random_state=42),
    param_distributions=param_dist,
    n_iter=30,
    cv=cv,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=2,
    random_state=42
)

random_search.fit(X, y)

# =============================================================================
# 4. RESULTS
# =============================================================================
print(f"\n{'='*70}")
print("HYPERPARAMETER TUNING RESULTS")
print(f"{'='*70}")

print(f"\nBest parameters found:")
for param, value in random_search.best_params_.items():
    print(f"  {param}: {value}")

print(f"\nBest CV ROC-AUC: {random_search.best_score_:.4f}")

# =============================================================================
# 5. OVERFITTING ANALYSIS
# =============================================================================
print(f"\n{'='*70}")
print("OVERFITTING ANALYSIS")
print(f"{'='*70}")

model = random_search.best_estimator_

y_pred_train = model.predict_proba(X)[:, 1]
train_auc = roc_auc_score(y, y_pred_train)
cv_auc = random_search.best_score_
gap = train_auc - cv_auc

print(f"\nTraining ROC-AUC: {train_auc:.4f}")
print(f"Cross-Validation ROC-AUC: {cv_auc:.4f}")
print(f"Gap (Train - CV): {gap:.4f}")

if gap < 0.10:
    print("✓ Good! Acceptable overfitting")
else:
    print("⚠ Warning! Significant overfitting")


EXPERIMENT: gradient_boosting_conservative

Running RandomizedSearchCV (trying 30 combinations)...
This may take 15-25 minutes...

Fitting 5 folds for each of 30 candidates, totalling 150 fits

HYPERPARAMETER TUNING RESULTS

Best parameters found:
  subsample: 0.7
  n_estimators: 200
  min_samples_split: 50
  min_samples_leaf: 20
  max_features: 0.5
  max_depth: 5
  learning_rate: 0.1

Best CV ROC-AUC: 0.9028

OVERFITTING ANALYSIS

Training ROC-AUC: 0.9686
Cross-Validation ROC-AUC: 0.9028
Gap (Train - CV): 0.0658
✓ Good! Acceptable overfitting


In [5]:
# =============================================================================
# 4. GENERATE PREDICTIONS
# =============================================================================
test_proba = model.predict_proba(X_test)[:, 1]

In [6]:
# =============================================================================
# 5. CREATE SUBMISSION FILE
# =============================================================================
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
submission_filename = f"../../outputs/predictions/gradient_boosting/{EXPERIMENT_NAME}_{timestamp}.csv"

submission = pd.DataFrame({
    'icustay_id': test_ids,
    'prediction': test_proba
})
submission.to_csv(submission_filename, index=False)
print(f"\n✓ Submission saved: {submission_filename}")


✓ Submission saved: ../../outputs/predictions/gradient_boosting/gradient_boosting_conservative_20251128_2255.csv


In [7]:
experiment_log = {
    'timestamp': timestamp,
    'experiment_name': EXPERIMENT_NAME,
    'model_type': 'GradientBoostingClassifier',
    'cv_roc_auc_mean': cv_auc,
    'train_roc_auc': train_auc,
    'overfitting_gap': gap,
    'kaggle_public_score': None,
    'kaggle_private_score': None,
    'parameters': str(random_search.best_params_),
    'description': MODEL_DESCRIPTION,
    'submission_file': submission_filename
}

log_file = '../experiment_log.csv'
log_df = pd.DataFrame([experiment_log])

import os
if os.path.exists(log_file):
    log_df.to_csv(log_file, mode='a', header=False, index=False)
else:
    log_df.to_csv(log_file, index=False)

In [8]:
# =============================================================================
# 7. SAVE MODEL (OPTIONAL)
# =============================================================================
import pickle


model_filename = f"../../outputs/models/{EXPERIMENT_NAME}_{timestamp}.pkl"
with open(model_filename, 'wb') as f:
    pickle.dump(model, f)
print(f"✓ Model saved: {model_filename}")

print(f"\n{'='*70}")
print("EXPERIMENT COMPLETE!")
print(f"{'='*70}")
print(f"\nNext steps:")
print(f"1. Upload {submission_filename} to Kaggle")
print(f"2. Record Kaggle score in {log_file}")
print(f"3. Copy this notebook and modify for next experiment")

✓ Model saved: ../../outputs/models/gradient_boosting_conservative_20251128_2255.pkl

EXPERIMENT COMPLETE!

Next steps:
1. Upload ../../outputs/predictions/gradient_boosting/gradient_boosting_conservative_20251128_2255.csv to Kaggle
2. Record Kaggle score in ../experiment_log.csv
3. Copy this notebook and modify for next experiment
